In [ ]:
! pip install -q -U datasets
! pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.30.2 trl==0.4.7 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 118.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset('b-mc2/sql-create-context')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 78577
    })
})

In [ ]:
print(dataset["train"]["question"][0])

How many heads of the departments are older than 56 ?


In [ ]:
dataset["train"]["context"][0]

'CREATE TABLE head (age INTEGER)'

In [ ]:
dataset["train"]["answer"][0]

'SELECT COUNT(*) FROM head WHERE age > 56'

In [ ]:
import pandas as pd
from datasets import Dataset

df=Dataset.to_pandas(dataset["train"])
df.head()

,question,context,answer
0,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B..."
2,"List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM..."
3,What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...


In [ ]:
len(df)

78577

In [ ]:
data=df.copy()
data.head()

,question,context,answer
0,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B..."
2,"List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM..."
3,What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...


In [ ]:
# instruction finetuning data preparation function

def prepare_dataset(df):
    text_col = []
    instruction = """you are a ai assitant who will solve the sql problem after reading the question on the base of context """ # change instuction according to the task

    for _ , row in df.iterrows():
        question = row["question"]
        answer = row["answer"]
        context=row["context"]
        text = ("### Instruction: \n" + instruction +
                "\n### question: \n" + question +
                "\n### context: \n" + context +
                "\n### answer :\n" + answer) # keeping output column in training dataset
        text_col.append(text)
    df.loc[:,'text'] = text_col
    return df

In [ ]:
train_df = prepare_dataset(data)

In [ ]:
train_df.head()

,question,context,answer,text
0,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,### Instruction: \nyou are a ai assitant who w...
1,"List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...",### Instruction: \nyou are a ai assitant who w...
2,"List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...",### Instruction: \nyou are a ai assitant who w...
3,What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...",### Instruction: \nyou are a ai assitant who w...
4,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,### Instruction: \nyou are a ai assitant who w...


In [ ]:
train_df["text"][0]

'### Instruction: \nyou are a ai assitant who will solve the sql problem after reading the question on the base of context \n### question: \nHow many heads of the departments are older than 56 ?\n### context: \nCREATE TABLE head (age INTEGER)\n### answer :\nSELECT COUNT(*) FROM head WHERE age > 56'

In [ ]:
train_dataset = Dataset.from_pandas(train_df)

In [ ]:
train_dataset

Dataset({
    features: ['question', 'context', 'answer', 'text'],
    num_rows: 78577
})

In [ ]:
train_dataset = train_dataset.train_test_split(test_size=0.2)

In [ ]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer', 'text'],
        num_rows: 62861
    })
    test: Dataset({
        features: ['question', 'context', 'answer', 'text'],
        num_rows: 15716
    })
})

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# sharded model path in hugging face
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

#model_name = 'NousResearch/Llama-2-7b-hf'

#MODEL_NAME = "OpenAssistant/falcon-7b-sft-mix-2000"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00014.bin:   0%|          | 0.00/981M [00:00<?, ?B/s]

pytorch_model-00002-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00003-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.05
lora_r = 8 # rank

# Parameter efficient finetuning for LoRA configuration

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules= ["q_proj","v_proj"], # we will only create adopters for q, v metrices of attention module
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:

import transformers

training_arguments = transformers.TrainingArguments(
        output_dir="llama2_qlora_finetuned",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        max_steps=100,
        fp16=True,
        push_to_hub=False
    )

In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset["train"],
        peft_config=peft_config, # passing peft config
        dataset_text_field="text", # mentioned the required column
        args=training_arguments, # training agruments
        tokenizer=tokenizer, # tokenizer
        packing=False,
        max_seq_length=512
    )

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

In [ ]:
# upcasting the layer norms in float 32 for more stable training

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,139.916600
20,51.284400
30,1.542800
40,61.030600
50,2.596100
60,14.222900
70,0.574900
80,5.227800
90,4.634800
100,0.618200


TrainOutput(global_step=100, training_loss=28.164917340278624, metrics={'train_runtime': 1101.6336, 'train_samples_per_second': 1.452, 'train_steps_per_second': 0.091, 'total_flos': 4472012280102912.0, 'train_loss': 28.164917340278624, 'epoch': 0.03})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
text = """
### Instruction:
Answer the following question based on the context
### question:
Which team had a year entering the league under 2009, located in Bath?
### context:
CREATE TABLE table_name_21 (team VARCHAR, year_entered_league VARCHAR, location VARCHAR)
### answer :"""

print(text)


### Instruction: 
Answer the following question based on the context 
### question: 
Which team had a year entering the league under 2009, located in Bath?
### context: 
CREATE TABLE table_name_21 (team VARCHAR, year_entered_league VARCHAR, location VARCHAR)
### answer :


In [ ]:
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=100 ,repetition_penalty=1.2)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkp


### Instruction: 
Answer the following question based on the context 
### question: 
Which team had a year entering the league under 2009, located in Bath?
### context: 
CREATE TABLE table_name_21 (team VARCHAR, year_entered_league VARCHAR, location VARCHAR)
### answer : 
Bath United


In [ ]:
train_dataset["test"]["answer"][0]

'SELECT MAX(wins) FROM table_name_73 WHERE goals_against = 37 AND ties > 0'